In [1]:
from keras import objectives, backend as K
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, LSTM, RepeatVector, TimeDistributed
from keras.models import Model
import keras

Using TensorFlow backend.
/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class VAE(object):
    def create(self, vocab_size=500, max_length=300, latent_rep_size=200):
        self.encoder = None
        self.decoder = None
        self.sentiment_predictor = None
        self.autoencoder = None

        x = Input(shape=(max_length,))
        x_embed = Embedding(vocab_size, 64, input_length=max_length)(x)

        vae_loss, encoded = self._build_encoder(x_embed, latent_rep_size=latent_rep_size, max_length=max_length)
        self.encoder = Model(inputs=x, outputs=encoded)

        encoded_input = Input(shape=(latent_rep_size,))
        predicted_sentiment = self._build_sentiment_predictor(encoded_input)
        self.sentiment_predictor = Model(encoded_input, predicted_sentiment)

        decoded = self._build_decoder(encoded_input, vocab_size, max_length)
        self.decoder = Model(encoded_input, decoded)

        self.autoencoder = Model(inputs=x, outputs=[self._build_decoder(encoded, vocab_size, max_length), self._build_sentiment_predictor(encoded)])
        self.autoencoder.compile(optimizer='Adam',
                                 loss=[vae_loss, 'binary_crossentropy'],
                                 metrics=['accuracy'])
        
    def _build_encoder(self, x, latent_rep_size=200, max_length=300, epsilon_std=0.01):
        h = Bidirectional(LSTM(500, return_sequences=True, name='lstm_1'), merge_mode='concat')(x)
        h = Bidirectional(LSTM(500, return_sequences=False, name='lstm_2'), merge_mode='concat')(h)
        h = Dense(435, activation='relu', name='dense_1')(h)

        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev=epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation='linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation='linear')(h)
    
        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))

    def _build_decoder(self, encoded, vocab_size, max_length):
        repeated_context = RepeatVector(max_length)(encoded)
    
        h = LSTM(500, return_sequences=True, name='dec_lstm_1')(repeated_context)
        h = LSTM(500, return_sequences=True, name='dec_lstm_2')(h)
    
        decoded = TimeDistributed(Dense(vocab_size, activation='softmax'), name='decoded_mean')(h)
    
        return decoded

    def _build_sentiment_predictor(self, encoded):
        h = Dense(100, activation='linear')(encoded)
    
        return Dense(1, activation='sigmoid', name='pred')(h)



In [4]:
from keras.callbacks import ModelCheckpoint
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
# from model import VAE
import numpy as np
import os

In [5]:
MAX_LENGTH = 300
NUM_WORDS = 1000

In [6]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

print("Training data")
print(X_train.shape)
print(y_train.shape)

print("Number of words:")
print(len(np.unique(np.hstack(X_train))))

Training data
(25000,)
(25000,)
Number of words:
998


In [7]:
X_train = pad_sequences(X_train, maxlen=MAX_LENGTH)
X_test = pad_sequences(X_test, maxlen=MAX_LENGTH)

train_indices = np.random.choice(np.arange(X_train.shape[0]), 2000, replace=False)
test_indices = np.random.choice(np.arange(X_test.shape[0]), 1000, replace=False)

X_train = X_train[train_indices]
y_train = y_train[train_indices]

X_test = X_test[test_indices]
y_test = y_test[test_indices]

In [8]:
temp = np.zeros((X_train.shape[0], MAX_LENGTH, NUM_WORDS))
temp[np.expand_dims(np.arange(X_train.shape[0]), axis=0).reshape(X_train.shape[0], 1), np.repeat(np.array([np.arange(MAX_LENGTH)]), X_train.shape[0], axis=0), X_train] = 1

X_train_one_hot = temp

temp = np.zeros((X_test.shape[0], MAX_LENGTH, NUM_WORDS))
temp[np.expand_dims(np.arange(X_test.shape[0]), axis=0).reshape(X_test.shape[0], 1), np.repeat(np.array([np.arange(MAX_LENGTH)]), X_test.shape[0], axis=0), X_test] = 1

x_test_one_hot = temp

In [9]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + \
               model_name + "-{epoch:02d}-{val_decoded_mean_acc:.2f}-{val_pred_loss:.2f}.h5"
    directory = os.path.dirname(filepath)

    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

    checkpointer = ModelCheckpoint(filepath=filepath,
                                   verbose=1,
                                   save_best_only=False)

    return checkpointer

In [10]:
def train():
    model = VAE()
    model.create(vocab_size=NUM_WORDS, max_length=MAX_LENGTH)

    checkpointer = create_model_checkpoint('models', 'rnn_ae')

    model.autoencoder.fit(x=X_train, y={'decoded_mean': X_train_one_hot, 'pred': y_train},
                          batch_size=10, epochs=10, callbacks=[checkpointer],
                          validation_data=(X_test, {'decoded_mean': x_test_one_hot, 'pred':  y_test}))

In [11]:
train()

Train on 2000 samples, validate on 1000 samples
Epoch 1/10
2000/2000 [==============================] - 4053s 2s/step - loss: 2.1551 - decoded_mean_loss: 1.4606 - pred_loss: 0.6946 - decoded_mean_acc: 0.3417 - pred_acc: 0.5340 - val_loss: 2.0256 - val_decoded_mean_loss: 1.3859 - val_pred_loss: 0.6397 - val_decoded_mean_acc: 0.3788 - val_pred_acc: 0.6310

Epoch 00001: saving model to models/rnn_ae-01-0.38-0.64.h5
Epoch 2/10
2000/2000 [==============================] - 3964s 2s/step - loss: 1.9697 - decoded_mean_loss: 1.4117 - pred_loss: 0.5581 - decoded_mean_acc: 0.3519 - pred_acc: 0.7165 - val_loss: 1.9938 - val_decoded_mean_loss: 1.3888 - val_pred_loss: 0.6050 - val_decoded_mean_acc: 0.3788 - val_pred_acc: 0.6620

Epoch 00002: saving model to models/rnn_ae-02-0.38-0.60.h5
Epoch 3/10
2000/2000 [==============================] - 3974s 2s/step - loss: 1.8889 - decoded_mean_loss: 1.4372 - pred_loss: 0.4517 - decoded_mean_acc: 0.3491 - pred_acc: 0.7995 - val_loss: 1.9430 - val_decoded_mean

In [27]:
from keras.models import load_model


model = VAE()
model.create(vocab_size=NUM_WORDS, max_length=MAX_LENGTH)
model.autoencoder.load_weights('models/rnn_ae-10-0.42-0.66.h5')
encoded = model.encoder.predict(X_train[np.newaxis, 0])

In [36]:
model.sentiment_predictor.predict(encoded)

array([[0.48782405]], dtype=float32)

In [30]:
decoded = model.decoder.predict(encoded)
decoded

array([[[0.00099968, 0.00100005, 0.00099999, ..., 0.00100016,
         0.00100017, 0.00099951],
        [0.00099921, 0.00100003, 0.00099999, ..., 0.00100045,
         0.00100037, 0.00099879],
        [0.00099868, 0.00099993, 0.00099999, ..., 0.00100083,
         0.00100054, 0.00099798],
        ...,
        [0.0009954 , 0.00099747, 0.00100081, ..., 0.00100491,
         0.001     , 0.00099422],
        [0.0009954 , 0.00099747, 0.00100081, ..., 0.00100491,
         0.001     , 0.00099422],
        [0.0009954 , 0.00099747, 0.00100081, ..., 0.00100491,
         0.001     , 0.00099422]]], dtype=float32)

In [33]:
np.argmax(decoded, axis=1)

array([[ 0,  0, 59,  0,  0,  0, 59, 41, 21,  0, 27, 41,  0, 59, 21,  4,
         6, 16, 59,  0,  0,  0,  0, 20,  0,  0,  5, 60,  1, 11, 59,  0,
        51,  0, 41,  3, 22,  0,  0, 58,  0,  0, 25,  4, 60,  6,  0,  3,
         2,  0, 26, 49,  0,  0,  0, 61, 51, 16,  0, 49, 27,  1, 59,  0,
        41, 21,  0,  1,  0,  1,  0, 45,  0, 26, 30, 36, 49, 25,  8, 32,
         3,  9, 17,  0, 11,  0,  0,  0, 50, 12,  0, 58,  0, 19, 79,  0,
        54, 49,  5, 11,  0,  0, 10, 32,  3, 39, 20, 61,  0, 22,  0,  0,
         0,  0, 58, 59, 35, 25, 61, 23, 21, 56,  4, 56, 61,  0,  5,  0,
         0,  0, 14,  0,  0, 60,  1,  0,  1,  0,  0,  0,  0, 49,  0,  0,
         4,  0, 59,  3,  5,  0, 50, 57, 19, 37,  0, 50,  2,  0,  0, 53,
         0,  2, 22,  0,  0,  0, 55,  0,  2,  0,  1,  5,  0, 26,  7,  0,
         0,  2,  0, 21, 60, 45,  0, 10,  0,  0,  0,  0,  0,  0,  0, 60,
         0, 13,  9, 34, 47, 25,  0, 11, 60, 36, 13,  0, 20,  0,  2, 15,
         0, 61, 17,  0,  8, 20,  0,  7, 16, 25,  0, 32, 49, 51, 